# AoE Playerbase Analysation

## TODO

- Normalise some and split up into separately stored tables
- Long-term: Create unique player profiles for the pseudonymized players
    - only store relevant data to track player activity
    - probably derive the rest from it
- 

In [460]:
import pandas as pd

from .scripts.util.common import GLOBAL_SETTINGS
from .scripts.util.decorators import with_temporary_dataframe
from .scripts.util import initialise_defaultdict_recursive, get_temp_df

df = pd.read_parquet("../data/dataframe.br.parq", engine="pyarrow")

ImportError: attempted relative import with no known parent package

### Set global vars

In [442]:
# TODO: When we iterate over the data we should set this always to the datas timestamp
# that we are working with
DATA_SET_TIMESTAMP = df['timestamp'].max()

# Dataframe for our output to JSON
DF_OUT = pd.DataFrame()
DF_OUT["timestamp"] = [DATA_SET_TIMESTAMP]
DF_OUT.set_index("timestamp", inplace=True)

# Mask for only having the last timestamp
LAST_TIMESTAMP = df['timestamp'] == df['timestamp'].max()

# Latest dataset
lt_df=df[LAST_TIMESTAMP]

activity_filters_lt = pd.DataFrame()

for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
    activity_filters_lt[f"{activity_period}d"] = ((lt_df['timestamp']-lt_df['last_match']) < pd.Timedelta(activity_period, "d"))

# Create a dict with the activity periods for game and leaderboards
ACTIVITY_GL_GROUPS = {}

for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
   ACTIVITY_GL_GROUPS[f"{activity_period}d"] = lt_df[activity_filters_lt[f"{activity_period}d"]].groupby(["game", "leaderboard"])

# Create a dict with the activity periods for games
ACTIVITY_GAME_GROUPS = {}

# TODO: Think through if this really does what it should:
# It groups by the game, later we use the `profile_id` field,
# which can exist multiple times in this group, we make it unique
# by using value_counts on it and count it as one profile if in the activity span
# This should be essentially what it should be.
for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
   ACTIVITY_GAME_GROUPS[f"{activity_period}d"] = lt_df[activity_filters_lt[f"{activity_period}d"]].groupby(["game"])

# Create a dict with the activity periods for profiles
ACTIVITY_PROFILE_GROUPS = {}

for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
   ACTIVITY_PROFILE_GROUPS[f"{activity_period}d"] = lt_df[activity_filters_lt[f"{activity_period}d"]].groupby(["profile_id"])
    

## Output DataFrame

In [443]:
DF_OUT

""
timestamp
2022-01-31 08:26:48.290235


In [444]:
ACTIVITY_GL_GROUPS
ACTIVITY_GAME_GROUPS
ACTIVITY_PROFILE_GROUPS["1d"].head()

,timestamp,game,leaderboard,steam_id,profile_id,rank,rating,highest_rating,previous_rating,country_code,name,known_name,num_games,streak,num_wins,win_percent,rating24h,games24h,wins24h,last_match
2478285,2022-01-31 08:26:48.290235,aoe2,rm,"b'\x14e\xc6""\xf6\xc2U.\xeb\x96S1@\xe0JSLS\x96\...",b'\xd1c\xaf\xef\xbfev\x88\xff\x8c\xe1HU\xccl\x...,6,2469,2485.0,2465.0,BE,"b""\xf7}HA1\xb6\xc3\xc82:\x1c'y\xc8r\xd1\x0e\x1...",b'n\xc6K\xcd\xc5\x98\xfen-\xb2p?\xf8\xc5\xbd\x...,3315,17,1936,58,4,1,1,2022-01-30 13:33:08
2478287,2022-01-31 08:26:48.290235,aoe2,rm,b'*(\x9fP\xf7F\xcc*\x8a\x15\x93%\xce)4zU\xde\x...,b'N.\x1e\x81\x05\x17Bi\xd1]\x9337V\xdd\x1ce\x9...,8,2462,2470.0,2458.0,PL,b'\x1a\x88u\xb5\xc2E\x0e{\xef\xce\xb2c\xf1\xd9...,b'n\xc6K\xcd\xc5\x98\xfen-\xb2p?\xf8\xc5\xbd\x...,1805,1,1118,62,4,1,1,2022-01-30 15:40:32
2478297,2022-01-31 08:26:48.290235,aoe2,rm,b'\x17~\x8a\xad\x89*\xdf#\xbd\xfe\xa0<\xfa\xaf...,b'\xb2;\xc4*\xce_\xc1\x02z\x88\xc7.\xcf\x9c\xf...,18,2425,2506.0,2423.0,BR,b'\xb7`\x91\x04[\xc0cDVQ\xb3\x99\xba\xee\xd2\x...,b'n\xc6K\xcd\xc5\x98\xfen-\xb2p?\xf8\xc5\xbd\x...,1150,9,827,72,11,4,4,2022-01-31 00:08:53
2478300,2022-01-31 08:26:48.290235,aoe2,rm,b'a[E\xa4\xdf\xd9\x1a\xef\x9cv\xfc[\xa0J#\t\xc...,b'\x9b\xbb\x95\xe8\x8e\t\xf5C1b\\\xf9\x16\x83[...,21,2412,2498.0,2405.0,RU,b']\xbar\xfd4tL\x93\xe2=\xbc\xc1()=!\x02\x13\x...,b'n\xc6K\xcd\xc5\x98\xfen-\xb2p?\xf8\xc5\xbd\x...,991,2,658,66,7,2,2,2022-01-30 11:48:50
2478310,2022-01-31 08:26:48.290235,aoe2,rm,"b',\x95\x85M\x9e\xd5\xf3\xb7\x8e9a\xb7\xdd\xd4...",b'\xb6{A\x97I\xbc\x19\xbb\xb5\xe1\xd0\xb4$\xd5...,31,2355,2402.0,2365.0,ES,b'\xf50\x9d\xbe\x85-\xea\x81\x99\x0f\xd8\xb2\x...,b'n\xc6K\xcd\xc5\x98\xfen-\xb2p?\xf8\xc5\xbd\x...,943,-1,598,63,-10,1,0,2022-01-30 11:18:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095288,2022-01-31 08:26:48.290235,aoe4,qm_4v4,b';\xd9\x12S\xcbC\xdc\x82<\xff\x8b\xec\xce\x8e...,b'\xd9\x03\x1e/\x18iC\x93\xc9\x1d\x1b\x10\xde0...,44729,413,827.0,429.0,None,b'\xbc\x95\xfa\x18\x0e\xc3\xf6\xac\xccZ3\xa1\x...,None,101,-1,34,34,-3,2,1,2022-01-30 16:24:23
3095289,2022-01-31 08:26:48.290235,aoe4,qm_4v4,None,b'z\xc1\x11gl\xe8\x15\x8a\x13\xff\x0c\x91{\xeb...,44730,410,857.0,425.0,None,b'2\t\xcd!\xc7=\x92\x96\xcd\xdc\xc5>\x86\x19\x...,None,349,-3,129,37,-26,10,3,2022-01-31 02:00:01
3095290,2022-01-31 08:26:48.290235,aoe4,qm_4v4,b'\x19\xe0Y1^&FZ\xdc\xb9ez#\xbd\xeeAuIn\xf5\xe...,b'{\xb3\x0e\x0fs\x16z\x81\xb02\xe3>\xaa\x03\xa...,44731,408,865.0,392.0,None,b'\xd2\x13\xec\t\xaf`\x0b\x8c\\;`]d\x03i\x80|\...,None,169,1,65,38,16,1,1,2022-01-30 16:24:23
3095291,2022-01-31 08:26:48.290235,aoe4,qm_4v4,b'\xe9\x18\x98\xe7svZ\xdb\x91\xa0\x04\x81\x0bd...,"b""\xca\xf2\xc3i\xd8`V\xe5'\xaf7\x02\x05 \x83\x...",44732,406,993.0,421.0,None,b'U\xcf1\xf7\xf4\x99\x85\xf22\x87_\x11\x7f\xf5...,None,208,-2,79,38,-47,6,1,2022-01-30 14:18:16


### Some information about the data

In [445]:
df.columns

Index(['timestamp', 'game', 'leaderboard', 'steam_id', 'profile_id', 'rank',
       'rating', 'highest_rating', 'previous_rating', 'country_code', 'name',
       'known_name', 'num_games', 'streak', 'num_wins', 'win_percent',
       'rating24h', 'games24h', 'wins24h', 'last_match'],
      dtype='object')

In [446]:
df.groupby(["timestamp"]).size()

timestamp
2022-01-27 22:17:09.156189    620819
2022-01-28 18:16:09.062004    618489
2022-01-29 19:24:12.630533    619666
2022-01-30 00:42:39.365196    619306
2022-01-31 08:26:48.290235    617015
dtype: int64

In [447]:
# DF size
df.shape
# Normal information
gl_lt_df.describe()

rank                                                 \
                        count     mean           std  min       25%      50%   
game leaderboard                                                               
aoe2 ew                1813.0    907.0    523.512337  1.0    454.00    907.0   
     rm               40045.0  20023.0  11560.140102  1.0  10012.00  20023.0   
     team_ew           2433.0   1217.0    702.490925  1.0    609.00   1217.0   
     team_rm          65119.0  32560.0  18798.380427  1.0  16280.50  32560.0   
     unranked        172525.0  86263.0  49803.821933  1.0  43132.00  86263.0   
aoe3 deathmatch          23.0     12.0      6.782330  1.0      6.50     12.0   
     supremacy_1v1     2987.0   1494.0    862.416953  1.0    747.50   1494.0   
     supremacy_team    2885.0   1443.0    832.972088  1.0    722.00   1443.0   
     treaty             613.0    307.0    177.102136  1.0    154.00    307.0   
aoe4 custom          130580.0  65290.5  37695.343413  1.0  32645.75  65290.5   
     qm_1v1           43623.0  21812.0  12593.019733  1.0  10906.50  21812.0   
     qm_2v2           56827.0  28414.0  16404.686211  1.0  14207.50  28414.0   
     qm_3v3           52807.0  26404.0  15244.212169  1.0  13202.50  26404.0   
     qm_4v4           44735.0  22368.0  12914.026483  1.0  11184.50  22368.0   

                                            rating               ... games24h  \
                           75%       max     count         mean  ...      75%   
game leaderboard                                                 ...            
aoe2 ew                1360.00    1813.0    1813.0  1002.599007  ...      0.0   
     rm               30034.00   40045.0   40045.0  1041.058884  ...      0.0   
     team_ew           1825.00    2433.0    2433.0  1010.758323  ...      0.0   
     team_rm          48839.50   65119.0   65119.0  1406.511571  ...      1.0   
     unranked        129394.00  172525.0  172525.0  1202.636690  ...      0.0   
aoe3 deathmatch          17.50      23.0      23.0  1294.391304  ...      1.0   
     supremacy_1v1     2240.50    2987.0    2987.0  1090.173418  ...      0.0   
     supremacy_team    2164.00    2885.0    2885.0  1252.786135  ...      0.0   
     treaty             460.00     613.0     613.0  1131.743883  ...      0.0   
aoe4 custom           97935.25  130580.0  130580.0   999.268724  ...      0.0   
     qm_1v1           32717.50   43623.0   43623.0  1002.117965  ...      0.0   
     qm_2v2           42620.50   56827.0   56827.0   998.244057  ...      0.0   
     qm_3v3           39605.50   52807.0   52807.0   996.145738  ...      0.0   
     qm_4v4           33551.50   44735.0   44735.0   996.263396  ...      0.0   

                            wins24h                                          \
                      max     count      mean       std  min  25%  50%  75%   
game leaderboard                                                              
aoe2 ew              28.0    1813.0  0.294539  0.987425  0.0  0.0  0.0  0.0   
     rm              40.0   40045.0  0.423149  1.129940  0.0  0.0  0.0  0.0   
     team_ew         21.0    2433.0  0.169749  0.608632  0.0  0.0  0.0  0.0   
     team_rm         40.0   65119.0  0.481733  1.167267  0.0  0.0  0.0  0.0   
     unranked         0.0  172525.0  0.000000  0.000000  0.0  0.0  0.0  0.0   
aoe3 deathmatch       7.0      23.0  0.521739  1.503619  0.0  0.0  0.0  0.0   
     supremacy_1v1   38.0    2987.0  0.598259  1.711815  0.0  0.0  0.0  0.0   
     supremacy_team  22.0    2885.0  0.438475  1.241661  0.0  0.0  0.0  0.0   
     treaty          15.0     613.0  0.213703  0.801352  0.0  0.0  0.0  0.0   
aoe4 custom          74.0  130580.0  0.229920  0.754152  0.0  0.0  0.0  0.0   
     qm_1v1          42.0   43623.0  0.428352  1.180704  0.0  0.0  0.0  0.0   
     qm_2v2          23.0   56827.0  0.273022  0.814523  0.0  0.0  0.0  0.0   
     qm_3v3          22.0   52807.0  0.241426  0.741464  0.0  0.0  0.0  0.0   
     qm_4v4          2

# Activity

## data_analyser.calculate_leaderboard_activity()

In [448]:
@with_temporary_dataframe
def calculate_leaderboard_activity(temp_df):
    for game, leaderboard, _, _, _, in GLOBAL_SETTINGS["LEADERBOARD_SETTINGS"]:
        for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
            try:
                temp_df[f"{activity_period}d_{game}_{leaderboard}_activity"]=ACTIVITY_GL_GROUPS[f"{activity_period}d"].get_group((game, leaderboard))["profile_id"].value_counts().sum()
            except (KeyError):
                temp_df[f"{activity_period}d_{game}_{leaderboard}_activity"]=None
    
    return temp_df


calculate_leaderboard_activity()

DF_OUT

,30d_aoe2_rm_activity,14d_aoe2_rm_activity,7d_aoe2_rm_activity,3d_aoe2_rm_activity,1d_aoe2_rm_activity,30d_aoe2_team_rm_activity,14d_aoe2_team_rm_activity,7d_aoe2_team_rm_activity,3d_aoe2_team_rm_activity,1d_aoe2_team_rm_activity,...,30d_aoe4_qm_3v3_activity,14d_aoe4_qm_3v3_activity,7d_aoe4_qm_3v3_activity,3d_aoe4_qm_3v3_activity,1d_aoe4_qm_3v3_activity,30d_aoe4_qm_4v4_activity,14d_aoe4_qm_4v4_activity,7d_aoe4_qm_4v4_activity,3d_aoe4_qm_4v4_activity,1d_aoe4_qm_4v4_activity
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-31 08:26:48.290235,40045,31559,24417,17699,10026,65119,52778,42194,31446,18393,...,52807,39533,29656,20656,10950,44735,33648,25643,18340,10208


## data_analyser.calculate_game_activity()

In [449]:
# Overall unique franchise player numbers of all leaderboards and games within the time period of X 

@with_temporary_dataframe
def calculate_game_activity(temp_df):
    for game in GLOBAL_SETTINGS["GAMES"]:
        for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
            try:
                temp_df[f"{activity_period}d_{game}_activity"]=ACTIVITY_GAME_GROUPS[f"{activity_period}d"][["profile_id"]].get_group(game).value_counts().sum()
            except (KeyError):
                temp_df[f"{activity_period}d_{game}_activity"]=None
    return temp_df
    
calculate_game_activity()
     
DF_OUT

,30d_aoe2_rm_activity,14d_aoe2_rm_activity,7d_aoe2_rm_activity,3d_aoe2_rm_activity,1d_aoe2_rm_activity,30d_aoe2_team_rm_activity,14d_aoe2_team_rm_activity,7d_aoe2_team_rm_activity,3d_aoe2_team_rm_activity,1d_aoe2_team_rm_activity,...,30d_aoe3_activity,14d_aoe3_activity,7d_aoe3_activity,3d_aoe3_activity,1d_aoe3_activity,30d_aoe4_activity,14d_aoe4_activity,7d_aoe4_activity,3d_aoe4_activity,1d_aoe4_activity
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-31 08:26:48.290235,40045,31559,24417,17699,10026,65119,52778,42194,31446,18393,...,6508,4917,3751,2686,1563,328571,237526,175292,123238,66361


## data_analyser.calculate_franchise_activity()

In [450]:
# Overall unique franchise player numbers of all leaderboards and games within the time period of X

@with_temporary_dataframe
def calculate_franchise_activity(temp_df):
    for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
        temp_df[f"{activity_period}d_franchise_activity"] = ACTIVITY_GL_GROUPS[f"{activity_period}d"][["profile_id"]].value_counts().sum()
    return temp_df

calculate_franchise_activity()

DF_OUT

,30d_aoe2_rm_activity,14d_aoe2_rm_activity,7d_aoe2_rm_activity,3d_aoe2_rm_activity,1d_aoe2_rm_activity,30d_aoe2_team_rm_activity,14d_aoe2_team_rm_activity,7d_aoe2_team_rm_activity,3d_aoe2_team_rm_activity,1d_aoe2_team_rm_activity,...,30d_aoe4_activity,14d_aoe4_activity,7d_aoe4_activity,3d_aoe4_activity,1d_aoe4_activity,30d_franchise_activity,14d_franchise_activity,7d_franchise_activity,3d_franchise_activity,1d_franchise_activity
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-31 08:26:48.290235,40045,31559,24417,17699,10026,65119,52778,42194,31446,18393,...,328571,237526,175292,123238,66361,534021,329877,247866,176567,97108


# Countries

## TODO: data_analyser.countries_per_leaderboard()

Question: How to store the data within the data frame? Or in general, store it.

In [451]:
# country_leaderboard_activity = initialise_defaultdict_recursive()

# for game, leaderboard, _, _, _, in GLOBAL_SETTINGS["LEADERBOARD_SETTINGS"]:
#     for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
#         try:
#             country_leaderboard_activity[f"{activity_period}d"][game][leaderboard]=ACTIVITY_GL_GROUPS[f"{activity_period}d"].get_group((game, leaderboard))["country_code"].value_counts(normalize=True, dropna=False).head(GLOBAL_SETTINGS["TOP_COUNTRIES"])
#         except (KeyError):
#             country_leaderboard_activity[f"{activity_period}d"][game][leaderboard]=None

# country_leaderboard_activity

s=ACTIVITY_GL_GROUPS[f"{activity_period}d"].get_group(("aoe2", "rm"))["country_code"].value_counts(normalize=True, dropna=False).head(GLOBAL_SETTINGS["TOP_COUNTRIES"])
s.T



US      0.112807
AR      0.089467
DE      0.084680
None    0.066028
FR      0.058847
ES      0.045881
GB      0.044285
MX      0.040794
TR      0.029523
CL      0.028326
BR      0.026032
CA      0.025534
AU      0.024337
TW      0.023339
CO      0.021145
RU      0.020048
CN      0.019449
NL      0.017754
IN      0.016657
CH      0.016457
IT      0.014063
BE      0.012368
PE      0.012168
CZ      0.010473
SE      0.010174
AT      0.009076
Name: country_code, dtype: float64

## TODO: data_analyser.countries_per_game()

In [452]:
# countries_game = initialise_defaultdict_recursive()

# for game in GLOBAL_SETTINGS["GAMES"]:
#     for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
#         try:
#             if isinstance(countries_game[f"{activity_period}d"][game], defaultdict):
#                 countries_game[f"{activity_period}d"][game]=0
#             countries_game[f"{activity_period}d"][game]=ACTIVITY_GAME_GROUPS[f"{activity_period}d"][["profile_id"]].get_group(game).value_counts(normalize=True, dropna=False).sum()
#         except (KeyError):
#             # happens because the unranked leaderboard has no activity
#             # we can pass this run
#             pass

# print(countries_game)

# gl_lt_df["country_code"].value_counts(normalize=True).head(25)
ACTIVITY_GAME_GROUPS["1d"][["game", "profile_id", "country_code"]].get_group("aoe2").value_counts(normalize=True)

game  profile_id                                                                                             country_code
aoe2  b"5'<\xbe\x80\xb1\xfb\x1a{\xd3\xeb\xd1AH\xb9Z\x8a\xa2\xa1\xea\xc4V}3b,\xa52"                           RU              0.000146
      b'\xd0\x93\xdbV\xc4\x86\x13\xec\xf7%\xa6\xd7c\xb7\xfe\t\xce\x97\xc2\x9e\xd5\x9c\x11\xee\x7f\x12<\xd6'  BR              0.000146
      b'\x0c\xeb\x8f\xbfQpj\xd0\xb4\xfb\xaab\xe7\x9f\xc6\x9au \n^\xd1\xe5\xef\x8d\xa1\xc6\x9e%'              CN              0.000146
      b"\xd4\xcfH\x9c\x15q'\xe8\xcc2\x87\x18\xabC\xc9X _\xe7|-N\xbd\xaa\xe7T\x01Q"                           IL              0.000146
      b'\xdbLZ\x1cO\xcc1\xc8s\x98V\xad\xf3\xf8\x11\xa6{\xc9\xae\xdd\xc8\xff\xbb|\x1et\xd2\xf5'               MX              0.000146
                                                                                                                               ...   
      b"\\\x8fTB\xe5\xb4\x0ef'\xfb\xff\x11\xfd\xd3\x0f\xd0\xbe\xc8d\\\xb1\

## TODO: data_analyser.countries_for_franchise()

In [453]:
# franchise_countries = {}
# for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
#     franchise_countries[f"{activity_period}d"] = ACTIVITY_GL_GROUPS[f"{activity_period}d"][["profile_id"]].value_counts(normalize=True)

# print(franchise_countries)
ACTIVITY_GL_GROUPS["30d"][["profile_id"]].value_counts(normalize=True)

game  leaderboard  profile_id                                                                                           
aoe2  ew           b'\xd0\xb2\xb1"b\xcc\xd3\xa5&\x004Z.\xf1[\x94C\xd1\xa3q\xeaT76\x91H\x97\x97'                             0.000552
                   b'\xd0\xae\xa1\x0c\x17a>ky\xc0\xd2\x7f\x1e\x14s\x1d\xc6nM\xfb\xe6!I\x1f\xc6w \x8d'                       0.000552
                   b'\xd0\x93\xdbV\xc4\x86\x13\xec\xf7%\xa6\xd7c\xb7\xfe\t\xce\x97\xc2\x9e\xd5\x9c\x11\xee\x7f\x12<\xd6'    0.000552
                   b"\xd0g8\x851\x86}\x02h\x0b\\\x15\xe4,P\xa1\x80\x90\xf8&\r\xfe\x13\xab\t\x08\x86'"                       0.000552
                   b'\xd0U\x1c\xcf\x9e\xa4P\xeb\xafX\xe7\xcc\xcdvd\x8fj4<E\xb8,\xb8\xe0\xbav\xd5\xbf'                       0.000552
                                                                                                                              ...   
aoe4  qm_4v4       b'U37\x81k\xfb\x8f\x14\xa1\xab\xe3\x9fF\x1c\xa3kYJ\xe0"\x8b\x1

# Platforms

## data_analyser.platforms_per_leaderboard()

In [454]:
# NOTE: We use max() here because it's unlikely that there will be ever be LESS THAN TWO Relic Link users as long
# as the franchise is active and this tool is running

@with_temporary_dataframe
def platforms_per_leaderboard(temp_df):
    for game, leaderboard, _, _, _, in GLOBAL_SETTINGS["LEADERBOARD_SETTINGS"]:
        for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
            try:
                relic=ACTIVITY_GL_GROUPS[f"{activity_period}d"].get_group((game, leaderboard))[["steam_id"]].value_counts(normalize=True, dropna=False).max()
                relic=round(relic,4)
                temp_df[f"{activity_period}d_{game}_{leaderboard}_platform_steam"]= 1-relic
                temp_df[f"{activity_period}d_{game}_{leaderboard}_platform_relic"]= relic
            except (KeyError):
                temp_df[f"{activity_period}d_{game}_{leaderboard}_platforms_steam"]= None
                temp_df[f"{activity_period}d_{game}_{leaderboard}_platforms_relic"]= None

    return temp_df

platforms_per_leaderboard()

DF_OUT

C:\Users\Jameson\AppData\Local\Temp\ipykernel_1356\1033798356.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_df[f"{activity_period}d_{game}_{leaderboard}_platform_steam"]= 1-relic
C:\Users\Jameson\AppData\Local\Temp\ipykernel_1356\1033798356.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_df[f"{activity_period}d_{game}_{leaderboard}_platform_relic"]= relic
C:\Users\Jameson\AppData\Local\Temp\ipykernel_1356\1033798356.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

C:\Users\Jameson\AppData\Local\Temp\ipykernel_1356\1033798356.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_df[f"{activity_period}d_{game}_{leaderboard}_platform_steam"]= 1-relic
C:\Users\Jameson\AppData\Local\Temp\ipykernel_1356\1033798356.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_df[f"{activity_period}d_{game}_{leaderboard}_platform_relic"]= relic
C:\Users\Jameson\AppData\Local\Temp\ipykernel_1356\1033798356.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

,30d_aoe2_rm_activity,14d_aoe2_rm_activity,7d_aoe2_rm_activity,3d_aoe2_rm_activity,1d_aoe2_rm_activity,30d_aoe2_team_rm_activity,14d_aoe2_team_rm_activity,7d_aoe2_team_rm_activity,3d_aoe2_team_rm_activity,1d_aoe2_team_rm_activity,...,30d_aoe4_qm_4v4_platform_steam,30d_aoe4_qm_4v4_platform_relic,14d_aoe4_qm_4v4_platform_steam,14d_aoe4_qm_4v4_platform_relic,7d_aoe4_qm_4v4_platform_steam,7d_aoe4_qm_4v4_platform_relic,3d_aoe4_qm_4v4_platform_steam,3d_aoe4_qm_4v4_platform_relic,1d_aoe4_qm_4v4_platform_steam,1d_aoe4_qm_4v4_platform_relic
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-31 08:26:48.290235,40045,31559,24417,17699,10026,65119,52778,42194,31446,18393,...,0.6883,0.3117,0.6978,0.3022,0.7001,0.2999,0.7067,0.2933,0.7105,0.2895


In [455]:
# Defragment after big insertion
DF_OUT = newframe = DF_OUT.copy()

## data_analyser.platforms_per_game()

In [456]:
# NOTE: We use max() here because it's unlikely that there will be ever be LESS THAN TWO Relic Link users as long
# as the franchise is active and this tool is running

@with_temporary_dataframe
def platforms_per_game(temp_df):
    for game in GLOBAL_SETTINGS["GAMES"]:
        for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
            try:
                relic=ACTIVITY_GAME_GROUPS[f"{activity_period}d"].get_group((game))[["steam_id"]].value_counts(normalize=True, dropna=False).max()
                relic=round(relic,4)
                temp_df[f"{activity_period}d_{game}_steam_platform"]=1-relic
                temp_df[f"{activity_period}d_{game}_relic_platform"]=relic

            except (KeyError):
                temp_df[f"{activity_period}d_{game}_steam_platform"]=None
                temp_df[f"{activity_period}d_{game}_relic_platform"]=None

    return temp_df

platforms_per_game()
                
DF_OUT

,30d_aoe2_rm_activity,14d_aoe2_rm_activity,7d_aoe2_rm_activity,3d_aoe2_rm_activity,1d_aoe2_rm_activity,30d_aoe2_team_rm_activity,14d_aoe2_team_rm_activity,7d_aoe2_team_rm_activity,3d_aoe2_team_rm_activity,1d_aoe2_team_rm_activity,...,30d_aoe4_steam_platform,30d_aoe4_relic_platform,14d_aoe4_steam_platform,14d_aoe4_relic_platform,7d_aoe4_steam_platform,7d_aoe4_relic_platform,3d_aoe4_steam_platform,3d_aoe4_relic_platform,1d_aoe4_steam_platform,1d_aoe4_relic_platform
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-31 08:26:48.290235,40045,31559,24417,17699,10026,65119,52778,42194,31446,18393,...,0.6617,0.3383,0.6695,0.3305,0.672,0.328,0.6751,0.3249,0.676,0.324


## data_analyser.platforms_for_franchise()

In [457]:
@with_temporary_dataframe
def platforms_for_franchise(temp_df):
    for activity_period in GLOBAL_SETTINGS["ACTIVITY_PERIODS"]:
        relic= ACTIVITY_GL_GROUPS[f"{activity_period}d"][["steam_id"]].value_counts(normalize=True, dropna=False).max()
        relic=round(relic,4)

        temp_df[f"{activity_period}d_franchise_steam_platform"]=1-relic
        temp_df[f"{activity_period}d_franchise_relic_platform"]=relic
    return temp_df

platforms_for_franchise()

DF_OUT

,30d_aoe2_rm_activity,14d_aoe2_rm_activity,7d_aoe2_rm_activity,3d_aoe2_rm_activity,1d_aoe2_rm_activity,30d_aoe2_team_rm_activity,14d_aoe2_team_rm_activity,7d_aoe2_team_rm_activity,3d_aoe2_team_rm_activity,1d_aoe2_team_rm_activity,...,30d_franchise_steam_platform,30d_franchise_relic_platform,14d_franchise_steam_platform,14d_franchise_relic_platform,7d_franchise_steam_platform,7d_franchise_relic_platform,3d_franchise_steam_platform,3d_franchise_relic_platform,1d_franchise_steam_platform,1d_franchise_relic_platform
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-31 08:26:48.290235,40045,31559,24417,17699,10026,65119,52778,42194,31446,18393,...,0.6307,0.3693,0.6399,0.3601,0.6416,0.3584,0.6445,0.3555,0.6427,0.3573


# Winrates

## Questions

1. does a country have a higher percentage of players in the upper percentiles?
    A: calculate the win rates by country and plot them to see if any are significantly above 50%

In [48]:
winrates = newest_df.groupby(["game", "leaderboard", "country_code"])

In [89]:
winrates["win_percent"].value_counts(normalize=True).loc[("aoe2","rm","US")].sort_values(ascending=False)

win_percent
50    0.179320
51    0.178854
52    0.092455
49    0.069399
48    0.047275
        ...   
77    0.000466
85    0.000466
91    0.000466
79    0.000233
90    0.000233
Name: win_percent, Length: 79, dtype: float64

In [100]:
winrates["win_percent"].agg(["median", "mean"])

median       mean
game leaderboard country_code                   
aoe2 ew          AL              53.0  49.666667
                 AR              51.0  50.906667
                 AT              54.0  55.454545
                 AU              52.0  53.300000
                 BD              51.0  51.000000
...                               ...        ...
aoe4 qm_4v4      VA              52.0  50.285714
                 VE              43.0  43.000000
                 VN              63.0  62.823529
                 ZA              52.5  49.966667
                 ZM              61.0  61.000000

[1398 rows x 2 columns]

In [101]:
winrates["win_percent"].apply(lambda x: x) # TODO

2478280    69
2478281    66
2478282    73
2478283    57
2478284    58
           ..
3095055    35
3095082    36
3095117    42
3095170    33
3095262    33
Name: win_percent, Length: 308495, dtype: int64

# Rating

## Questions

1. do more active players have a higher rating change over time